In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
import hardware_control.wx_programs as wx
import hardware_control.bnc as bnc
from classes.generator import *
from hardware_control.hardware_config import *
from experiment_configuration.values import *
from classes.qubit_class import *
import daq.daq_programs_homo as daq
import seaborn as sns
import standard_sequences.rabi as rabi
import analysis.plotting as plotting
import analysis.analysis as analysis
import standard_sequences.spectroscopy as spec
import standard_sequences.rabi as rabi
import standard_sequences.T1_measurements as T1
import standard_sequences.parametric_coupling as parametric_coupling
import qubit_nonH_cal as qnhc 

In [3]:
wx_addr = wx.get_wx_address()
bnc.set_bnc_output(
    general_vals_dict["qubit_bnc"], power_dBm=13, bnc_addr=target_bnc_address_6
)
bnc.set_bnc_output(
    readout_dict["RO_LO"],
    power_dBm=readout_dict["RO_LO_pwr"],
    bnc_addr=bnc_address["target_bnc_black"],
)


In [4]:
q1 = Qubit(q1_dict, readout_dict)
q2 = Qubit(q2_dict, readout_dict)
readout = Readout(readout_dict)
print(f"{q1}\n{q2}")
# bnc.set_DC_output(bnc_addr=C2_address, voltage=0.01)

Qubit(ro_amp=0.5, ef_half_amp=1.3, ge_time=77, ge_amp=1, RO_IF=None, RO_LO=6.6247, qubit_thr=[-10000, -600], ef_ssm=-0.25684, qubit_id=q1, RO_LO_pwr=16, ef_amp=1.5, ro_dur=3000, ge_ssm=-0.117, ef_time=44, IQ_angle=60, ef_half_time=25, RO_LO_45=6.3804, mixer_offset_ef=6, ROIF=0.1026600000000002, mixer_offset_ge=2.5, RO_LO_34=6.4804, ro_freq=6.72736)
Qubit(ro_amp=0.45, ef_half_amp=None, ge_time=62, ge_amp=1, RO_IF=None, RO_LO=6.6247, qubit_thr=[-10000, 1900], ef_ssm=-0.2962, qubit_id=q2, RO_LO_pwr=16, ef_amp=1, ro_dur=3000, ge_ssm=-0.155, ef_time=None, IQ_angle=25, ef_half_time=None, RO_LO_45=6.3804, mixer_offset_ef=None, ROIF=0.030739999999999768, mixer_offset_ge=2.5, RO_LO_34=6.4804, ro_freq=6.65544)


In [ ]:
num_steps = 101
sweep_time = 200
reps = 2000

# Run Rabi in ge manifold
ge_vals, ge_df, ge_rabi_amp, ge_pi_time = qnhc.run_rabi(
    q1, q2, general_vals_dict, num_steps, sweep_time, reps, manifold="ge"
)

# Run Rabi in ef manifold
ef_vals, ef_df, ef_rabi_amp, ef_pi_time = qhnc.run_rabi(
    q1, q2, general_vals_dict, num_steps, sweep_time, reps, manifold="ef"
)


# validating the amp and time parameters using gp minimize on a bang bang sequence in ge manifold
optimal_amp, optimal_time, result = qnhc.optimize_pi_pulse(
    q1, q2, general_vals_dict, reps, initial_amp, initial_time, manifold="ge"
)

print(f"Optimal amplitude: {optimal_amp}")
print(f"Optimal time: {optimal_time}")
print(f"Best score: {result.fun}")

print(f"Amplitude Difference: {np.abs(ge_rabi_amp - optimal_amp)}")
print(f"Time Difference: {np.abs(ge_pi_time - optimal_time)}")

q1.ge_time = optimal_time
q1.ge_amp = optimal_amp
q1.ef_time = optimal_time
q1.ef_amp = optimal_amp




In [ ]:
sweep_amp_center = 1
sweep_freq_center = -0.020
num_steps = 201
reps  = 2000
swap_optimal_freq, swap_optimal_time, swap_optimal_amp, swap_result = qnhc.optimize_swap_gate(q1, q2, general_vals_dict, reps, initial_freq, initial_amp)



